In [ ]:
import numpy as np
from pydrake.all import (
    DiagramBuilder,
    AddMultibodyPlantSceneGraph,
    Parser,
    MeshcatVisualizer,
    StartMeshcat,
    MeshcatVisualizerParams,
    Simulator,
    JointSliders,
    InverseKinematics,
    RotationMatrix,
    Solve,
    ContactVisualizerParams,
    ContactVisualizer,
    GeometrySet,
    CollisionFilterDeclaration,
    Role,
    eq,
    RigidTransform,
)
from manipulation import ConfigureParser, running_as_notebook
import pydot
from IPython.display import display, Image, SVG

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://7e656c9a-e35c-4740-b249-07fef330708d.deepnoteproject.com/7003/


## Problem Description
We will solve inverse kinematics for one arm on a PR2 robot. This robot model has been modified to have convex collision geometries, and to simplify this problem, we have welded (fixed) several joints that are irrelavent to the kinematics of the arms.

In [ ]:
def build_env():
    """Load in models and build the diagram."""
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl("package://manipulation/pr2_shelves.dmd.yaml")
    plant.Finalize()

    MeshcatVisualizer.AddToBuilder(
        builder,
        scene_graph.get_query_output_port(),
        meshcat,
        MeshcatVisualizerParams(delete_on_initialization_event=False),
    )

    diagram = builder.Build()
    return diagram, plant, scene_graph

In [ ]:
def filterCollsionGeometry(scene_graph, context=None):
    """Some robot models may appear to have self collisions due to overlapping collision geometries.
    This function filters out such problems for our PR2 model."""
    if context is None:
        filter_manager = scene_graph.collision_filter_manager()
    else:
        filter_manager = scene_graph.collision_filter_manager(context)
    inspector = scene_graph.model_inspector()

    pr2 = {}
    shelves = []
    tables = []

    for gid in inspector.GetGeometryIds(
        GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity
    ):
        gid_name = inspector.GetName(inspector.GetFrameId(gid))
        if "pr2" in gid_name:
            link_name = gid_name.split("::")[1]
            pr2[link_name] = [gid]

    def add_exclusion(set1, set2=None):
        if set2 is None:
            filter_manager.Apply(
                CollisionFilterDeclaration().ExcludeWithin(GeometrySet(set1))
            )
        else:
            filter_manager.Apply(
                CollisionFilterDeclaration().ExcludeBetween(
                    GeometrySet(set1), GeometrySet(set2)
                )
            )

    # Robot-to-self collisions
    add_exclusion(
        pr2["base_link"],
        pr2["l_shoulder_pan_link"]
        + pr2["r_shoulder_pan_link"]
        + pr2["l_upper_arm_link"]
        + pr2["r_upper_arm_link"]
        + pr2["head_pan_link"]
        + pr2["head_tilt_link"],
    )
    add_exclusion(
        pr2["torso_lift_link"], pr2["head_pan_link"] + pr2["head_tilt_link"]
    )
    add_exclusion(
        pr2["l_shoulder_pan_link"] + pr2["torso_lift_link"],
        pr2["l_upper_arm_link"],
    )
    add_exclusion(
        pr2["r_shoulder_pan_link"] + pr2["torso_lift_link"],
        pr2["r_upper_arm_link"],
    )
    add_exclusion(pr2["l_forearm_link"], pr2["l_gripper_palm_link"])
    add_exclusion(pr2["r_forearm_link"], pr2["r_gripper_palm_link"])

In [ ]:
goal_rotation = RotationMatrix(
    [
        [1, 0, 0],
        [0, -1, 0],
        [0, 0, -1],
    ]
)
goal_position = np.array([-0.83, 0.18, 1.4])
goal_pose = RigidTransform(goal_rotation, goal_position)

### IK for a Mobile Manipulator

Given a RigidTransform X_WG, compute a robot configuration placing the left gripper at that pose. We use optimization to solve the IK problem, and we repeatedly solve the program with random initializations until it succeeds. We have implemented a skeleton of the necessary code in the following function, but you must complete several pieces:

- Add position and orientation constraints to the gripper frame. The end effector should match the desired pose to within 1mm translation along each axis, and the rotation should be off by no more than 1 degree.
- If `fix_base` is True, constrain the base pose $(x,y,\theta)$ to be equal to `base_pose`.
- Add a collision free constraint with [AddMinimumDistanceLowerBoundConstraint](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_inverse_kinematics.html#a2ecd71efd675a7e1a4293adb05c9b9df). The minimum distance between any pair of collision geometries should be at least 1cm.
- Compute a random initial guess for the joint angles within the robot's joint limits. You can access the joint limits from the multibody plant, but some of the joints are angle-valued and don't have limits. For these joints, use the range $[-\pi,\pi]$.

In [ ]:
def solve_ik(X_WG, max_tries=10, fix_base=False, base_pose=np.zeros(3)):
    diagram, plant, scene_graph = build_env()

    gripper_frame = plant.GetFrameByName("l_gripper_palm_link")

    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)
    sg_context = scene_graph.GetMyContextFromRoot(context)
    filterCollsionGeometry(scene_graph, sg_context)

    # Note: passing in a plant_context is necessary for collision-free constraints!
    ik = InverseKinematics(plant, plant_context)
    q_variables = ik.q()  # Get variables for MathematicalProgram
    prog = ik.prog()  # Get MathematicalProgram
    q_nominal = np.zeros(len(q_variables))
    q_nominal[0:3] = base_pose
    prog.AddQuadraticErrorCost(
        np.eye(len(q_variables)), q_nominal, q_variables
    )

    # Add your constraints here
    world_frame = plant.world_frame()
    p_WG_lower = X_WG.translation() - 0.001*np.ones(3)
    p_WG_upper = X_WG.translation() - 0.001*np.ones(3)
    ik.AddPositionConstraint(
        frameA=world_frame,
        frameB=gripper_frame,
        p_BQ=np.zeros(3),
        p_AQ_lower=p_WG_lower,
        p_AQ_upper=p_WG_upper,
    )

    ik.AddOrientationConstraint(
            frameAbar=world_frame,
            R_AbarA=X_WG.rotation(),
            frameBbar=gripper_frame,
            R_BbarB=RotationMatrix(),
            theta_bound=np.deg2rad(1),
    )

    if fix_base:
        prog.AddConstraint(eq(q_variables[0:3], base_pose))

    ik.AddMinimumDistanceLowerBoundConstraint(0.01)

    for count in range(max_tries):
        # Compute a random initial guess here
        #rand_vals = (np.random.rand(len(q_variables)-3) - 0.5)*2*np.pi

        joint_lower_limits = plant.GetPositionLowerLimits()
        joint_upper_limits = plant.GetPositionUpperLimits()

        for i in range(len(joint_lower_limits)):
            if np.isinf(joint_lower_limits[i]):
                joint_lower_limits[i] = -np.pi

        for i in range(len(joint_upper_limits)):
            if np.isinf(joint_upper_limits[i]):
                joint_upper_limits[i] = np.pi

        rand_vals = np.random.rand(len(q_variables)) * (joint_upper_limits - joint_lower_limits) + joint_lower_limits
        prog.SetInitialGuess(q_variables[3:], rand_vals[3:])
        result = Solve(prog)

        if running_as_notebook:
            render_context = diagram.CreateDefaultContext()
            plant.SetPositions(
                plant.GetMyContextFromRoot(render_context),
                result.GetSolution(q_variables),
            )
            diagram.ForcedPublish(context)

        if result.is_success():
            print("Succeeded in %d tries!" % (count + 1))
            return result.GetSolution(q_variables)

    print("Failed!")
    return None

First, we show an example where we have fixed the base. It may take many tries for it to solve, or not solve at all! At each iteration, we visualize where the optimizer stopped, so you can see what the failures look like.

In [ ]:
solve_ik(
    goal_pose,
    max_tries=20,
    fix_base=True,
    base_pose=np.array([-1.23, 0.05, 0]),
)

Succeeded in 4 tries!


array([-1.23000000e+00,  5.00000000e-02,  0.00000000e+00,  2.62966277e-01,
       -3.85773430e-01, -5.10316608e-02,  3.09848893e-03,  0.00000000e+00,
       -1.53742416e-03,  0.00000000e+00, -6.63343752e-07,  2.12651592e-01,
       -1.78381081e-01,  4.95618161e-01, -1.62523584e+00, -1.59645960e+01,
       -1.67365477e+00,  5.74229331e+00])

When we allow the base to move freely, the inverse kinematics can be solved much more easily!

In [ ]:
solve_ik(goal_pose, fix_base=False)

Succeeded in 2 tries!


array([-1.07979290e+00, -2.13754303e-01, -4.23626269e+00,  3.10000000e-01,
       -2.59331799e-01, -7.36513698e-08,  2.57191063e-07,  0.00000000e+00,
       -3.62965911e-07,  0.00000000e+00,  4.21108835e-07, -7.14601837e-01,
       -4.28658210e-01,  1.33172241e+00, -9.75458246e-01,  7.41262015e-01,
       -4.79814771e-01,  5.15208653e-01])

In [ ]:
from manipulation.exercises.mobile.test_mobile_base_ik import (
    TestMobileBaseIk,
)
from manipulation.exercises.grader import Grader

Grader.grade_output([TestMobileBaseIk], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 5/5.

Score for Test solve_ik is 5/5.
- Succeeded in 1 tries!
Succeeded in 5 tries!
Succeeded in 1 tries!
Failed!



<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7e656c9a-e35c-4740-b249-07fef330708d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>